In [2]:
! pip install pandas

In [3]:
! pip install pymorphy3

Объясняю в чем суть нашего подхода. Вода это очеь субъективный критерий. Поэтому плодходитм с ммаксимальной осторожностью 

проверка на воду у нас будет состоять из двух этапов 

1) индекс читаемости - насколько текст читаемый

| Значение | Что значит  |
| -------- | ----------- |
| 80–100   | очень легко |
| 60–80    | нормально   |
| 40–60    | тяжеловато  |
| < 40     | сложно      |


2)readability_level - по сути тот же индекс читаемости только для нас с вами что бы на цифры не смотрели а сразу понимали каак читаеться текст 

3)stopword_rati - стоп слова показывает насколько текст структурный и смысловой или разговорный и «водянистый»

| Значение  | Интерпретация      |
| --------- | ------------------ |
| < 0.25    | плотный текст      |
| 0.25–0.35 | нормально          |
| > 0.4     | подозрение на воду |

4)adj_ratio - прилагаткльные описывает а не сообщает факты


| Значение  | Значение для воды |
| --------- | ----------------- |
| < 0.12    | фактология        |
| 0.12–0.18 | нейтрально        |
| > 0.2     | возможная вода    |


5)adv_ratio - наречия 


| Значение  | Значение           |
| --------- | ------------------ |
| < 0.03    | сухой текст        |
| 0.03–0.07 | нормально          |
| > 0.1     | эмоциональная вода |


6) repetition_ratio - max(count(word)) / total_words

Что показывает: повтор мыслей, зацикленность, «жевание» темы

| Значение | Значение |
| -------- | -------- |
| < 0.05   | хорошо   |
| 0.05–0.1 | терпимо  |
| > 0.15   | вода     |



In [4]:
import re
import string
import pandas as pd
import pymorphy3

morph = pymorphy3.MorphAnalyzer()


Функция считает количество гласных букв в слове. Мы упрощённо считаем: 1 гласная = 1 слог.

Как работает 

Возьмем каждую букву в слове,если это гласная, прибавим 1,а потом сложи все единицы».

In [ ]:
def count_syllables(word: str) -> int:
    vowels = 'аеёиоуыэюяАЕЁИОУЫЭЮЯ'
    count = 0

    for char in word:
        if char in vowels:
            count += 1

    return count



Берёт текст и возвращает 3 числа:

1. количество предложений

2. количество слов

3. общее количество слогов

In [25]:
def analyze_text_simple(text: str):
    raw_sentences = re.split(r'[.!?…]+', text)
    sentences = []

    for s in raw_sentences:
        s = s.strip()
        if s:
            sentences.append(s)

    words = re.findall(r'\b[а-яА-ЯёЁ]+\b', text)

    syllables = 0
    for word in words:
        normal_word = morph.parse(word)[0].normal_form
        syllables += count_syllables(normal_word)

    return len(sentences), len(words), syllables



Фозвращает возвращает

1. числовой индекс

2. текстовую категорию читаемости

In [26]:
def readability_index(text: str):
    sentences, words, syllables = analyze_text(text)

    if sentences == 0 or words == 0:
        return None, "ТЕКСТ СЛИШКОМ КОРОТКИЙ"

    index = 206.835 - 1.3 * (words / sentences) - 60.1 * (syllables / words)


    if index > 90:
        level = "ОЧЕНЬ ВЫСОКИЙ"
    elif index > 80:
        level = "ВЫСОКИЙ"
    elif index > 70:
        level = "ВЫШЕ СРЕДНЕГО"
    elif index > 60:
        level = "СРЕДНИЙ"
    elif index > 50:
        level = "НИЖЕ СРЕДНЕГО"
    elif index > 30:
        level = "НИЗКИЙ"
    else:
        level = "ОЧЕНЬ НИЗКИЙ"

    return round(index, 2), level


In [8]:
df = pd.read_csv('news.csv')
df


,source,title,text,publication_date,rubric,subrubric,tags
0,lenta.ru,Синий богатырь,В 1930-е годы Советский Союз охватила лихорадк...,2020-08-30T00:01:00+03:00,Экономика,Госэкономика,NaN
1,lenta.ru,Загитова согласилась вести «Ледниковый период»,Олимпийская чемпионка по фигурному катанию Ал...,2020-08-31T20:04:00+03:00,Спорт,Зимние виды,NaN
2,lenta.ru,Объяснена опасность однообразного питания,Российский врач-диетолог Римма Мойсенко объясн...,2020-08-31T20:07:00+03:00,Из жизни,Еда,NaN
3,lenta.ru,«Предохраняться? А зачем?»,В 2019 году телеканал «Ю» запустил адаптацию з...,2020-08-30T00:04:00+03:00,Интернет и СМИ,ТВ и радио,NaN
4,lenta.ru,Ефремов систематически употреблял наркотики,Актер Михаил Ефремов систематически употребл...,2020-08-31T18:27:00+03:00,Культура,Кино,NaN
...,...,...,...,...,...,...,...
21668,tjournal.ru,\n Россия прекратил...,\n\n \n \n \n \n \n...,1578056674,NaN,NaN,NaN
21669,tjournal.ru,\n Во Владивостоке ...,\n\n \n \n \n \n \n...,1577866951,NaN,NaN,NaN
21670,tjournal.ru,\n Дым от австралий...,\n\n \n \n \n \n \n...,1577864124,NaN,NaN,NaN
21671,tjournal.ru,\n Около 200 жителе...,\n\n \n \n \n \n \n...,1577894168,NaN,NaN,NaN


Ну тут все просто читаем весь текст и применяем к нему функцию readability_index. сохраняем уже новый датасет с индексом читемости 

In [9]:
df = pd.read_csv("news.csv")   
df = df[['text']]

df['readability_index'], df['readability_level'] = zip(
    *df['text'].apply(readability_index)
)


In [10]:
df

,text,readability_index,readability_level
0,В 1930-е годы Советский Союз охватила лихорадк...,39.57,НИЗКИЙ
1,Олимпийская чемпионка по фигурному катанию Ал...,42.97,НИЗКИЙ
2,Российский врач-диетолог Римма Мойсенко объясн...,39.74,НИЗКИЙ
3,В 2019 году телеканал «Ю» запустил адаптацию з...,53.83,НИЖЕ СРЕДНЕГО
4,Актер Михаил Ефремов систематически употребл...,48.80,НИЗКИЙ
...,...,...,...
21668,\n\n \n \n \n \n \n...,43.04,НИЗКИЙ
21669,\n\n \n \n \n \n \n...,50.18,НИЖЕ СРЕДНЕГО
21670,\n\n \n \n \n \n \n...,67.59,СРЕДНИЙ
21671,\n\n \n \n \n \n \n...,43.53,НИЗКИЙ


In [11]:
df.to_csv(
    "news_with_readability.csv",
    index=False,
    encoding="utf-8-sig"
)

In [12]:
df

,text,readability_index,readability_level
0,В 1930-е годы Советский Союз охватила лихорадк...,39.57,НИЗКИЙ
1,Олимпийская чемпионка по фигурному катанию Ал...,42.97,НИЗКИЙ
2,Российский врач-диетолог Римма Мойсенко объясн...,39.74,НИЗКИЙ
3,В 2019 году телеканал «Ю» запустил адаптацию з...,53.83,НИЖЕ СРЕДНЕГО
4,Актер Михаил Ефремов систематически употребл...,48.80,НИЗКИЙ
...,...,...,...
21668,\n\n \n \n \n \n \n...,43.04,НИЗКИЙ
21669,\n\n \n \n \n \n \n...,50.18,НИЖЕ СРЕДНЕГО
21670,\n\n \n \n \n \n \n...,67.59,СРЕДНИЙ
21671,\n\n \n \n \n \n \n...,43.53,НИЗКИЙ


In [13]:
! pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 791.7/791.7 kB 30.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [nltk]4/5 [nltk]]]


считаем стоп слова

In [15]:
import pandas as pd
import re
from collections import Counter
import pymorphy3
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

morph = pymorphy3.MorphAnalyzer()
ru_stopwords = set(stopwords.words("russian"))

df = pd.read_csv("news_with_readability.csv")


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/support/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


информативный текст → больше существительных, фактов, терминов

«вода» → много связок, вводных конструкций, стоп-слов




In [ ]:
def stopword_ratio_simple(text):
    # 1. получаем слова
    words = re.findall(r'\b[а-яА-ЯёЁ]+\b', text.lower())

    if len(words) == 0:
        return 0

    # 2. подсчёт стоп-слов берём каждое слово, если оно есть в списке ru_stopwords, считаем его как 1
    stopword_count = 0
    for word in words:
        if word in ru_stopwords:
            stopword_count += 1

    # 3. считаем долю
    ratio = stopword_count / len(words)
    return ratio


In [ ]:
def pos_ratios(text):
    words = re.findall(r'\b[а-яА-ЯёЁ]+\b', text) #находит все русские слова
    pos = Counter() # cоздаёт счётчик частей речи

    for w in words: # определяем часть речи каждого слова
        p = morph.parse(w)[0].tag.POS
        pos[p] += 1

    total = sum(pos.values())
    if total == 0:
        return 0, 0

    adj = pos.get("ADJF", 0) + pos.get("ADJS", 0)
    adv = pos.get("ADVB", 0)

    return adj / total, adv / total

df["adj_ratio"], df["adv_ratio"] = zip(
    *df["text"].apply(pos_ratios)
)


Функция вычисляет долю самого часто повторяющегося слова в тексте.

Это важно  так как повторение слов часто сигнализирует о воде — много пустых повторов, болтовня, SEO-тексты.

In [19]:
def repetition_ratio(text):
    words = re.findall(r'\b[а-яА-ЯёЁ]+\b', text.lower())
    if not words:
        return 0
    counts = Counter(words)
    return max(counts.values()) / len(words)

df["repetition_ratio"] = df["text"].apply(repetition_ratio)


In [20]:
df.to_csv(
    "news_with_features.csv",
    index=False,
    encoding="utf-8-sig"
)


In [ ]:
df


,text,readability_index,readability_level,stopword_ratio,adj_ratio,adv_ratio,repetition_ratio
0,В 1930-е годы Советский Союз охватила лихорадк...,39.57,НИЗКИЙ,0.281005,0.156463,0.047096,0.047096
1,Олимпийская чемпионка по фигурному катанию Ал...,42.97,НИЗКИЙ,0.309211,0.138158,0.019737,0.039474
2,Российский врач-диетолог Римма Мойсенко объясн...,39.74,НИЗКИЙ,0.246377,0.130435,0.086957,0.028986
3,В 2019 году телеканал «Ю» запустил адаптацию з...,53.83,НИЖЕ СРЕДНЕГО,0.342555,0.133197,0.049863,0.044057
4,Актер Михаил Ефремов систематически употребл...,48.80,НИЗКИЙ,0.238806,0.104478,0.059701,0.067164
...,...,...,...,...,...,...,...
21668,\n\n \n \n \n \n \n...,43.04,НИЗКИЙ,0.244898,0.132653,0.010204,0.061224
21669,\n\n \n \n \n \n \n...,50.18,НИЖЕ СРЕДНЕГО,0.207692,0.115385,0.023077,0.023077
21670,\n\n \n \n \n \n \n...,67.59,СРЕДНИЙ,0.241176,0.135294,0.052941,0.070588
21671,\n\n \n \n \n \n \n...,43.53,НИЗКИЙ,0.251073,0.195279,0.047210,0.047210


In [23]:
! pip install openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]


In [6]:
import pandas as pd

In [8]:
df = pd.read_excel('news_with_water.xlsx')

In [9]:
df.to_csv('news_with_water.csv', index=False)


In [10]:
import pandas as pd

df = pd.read_csv("news_with_water.csv")

print(df.shape)
print(df["water"].value_counts())
print(df.isnull().sum())

(3000, 8)
water
0    2835
1     165
Name: count, dtype: int64
text                 0
readability_index    0
readability_level    0
stopword_ratio       0
adj_ratio            0
adv_ratio            0
repetition_ratio     0
water                0
dtype: int64


In [11]:
import pandas as pd

df = pd.read_csv("news_with_water.csv")

df_water_0 = df[df['water'] == 0]
df_water_1 = df[df['water'] == 1]


df_water_0 = df_water_0.sample(n=835, random_state=42)

df_balanced = pd.concat([df_water_0, df_water_1])

print(df_balanced['water'].value_counts())


water
0    835
1    165
Name: count, dtype: int64


In [12]:
features = ["readability_index", "stopword_ratio", "adj_ratio", "adv_ratio", "repetition_ratio"]
X = df_balanced[features]  # Признаки
y = df_balanced['water']  

In [15]:
! pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 23.6 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 56.2 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-learn] [scikit-learn]


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


print(f"Train size: {len(X_train)} | Water=0: {y_train.value_counts()[0]}, Water=1: {y_train.value_counts()[1]}")
print(f"Test size: {len(X_test)} | Water=0: {y_test.value_counts()[0]}, Water=1: {y_test.value_counts()[1]}")


Train size: 800 | Water=0: 668, Water=1: 132
Test size: 200 | Water=0: 167, Water=1: 33


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [26]:
from sklearn.linear_model import LogisticRegression


model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)


model.fit(X_train_scaled, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,42
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [27]:
from sklearn.metrics import classification_report, roc_auc_score


y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)[:, 1]  


print(classification_report(y_test, y_pred))  
print("ROC AUC:", roc_auc_score(y_test, y_proba))  


              precision    recall  f1-score   support

           0       0.98      0.75      0.85       167
           1       0.42      0.91      0.57        33

    accuracy                           0.78       200
   macro avg       0.70      0.83      0.71       200
weighted avg       0.88      0.78      0.80       200

ROC AUC: 0.8684449283251678


In [28]:
import joblib

joblib.dump(model, "logreg_water_model.pkl")


['logreg_water_model.pkl']

In [29]:
model = joblib.load("logreg_water_model.pkl")

In [24]:
from sklearn.ensemble import RandomForestClassifier


rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_model.fit(X_train_scaled, y_train)


y_pred_rf = rf_model.predict(X_test_scaled)
y_proba_rf = rf_model.predict_proba(X_test_scaled)[:, 1]


print(classification_report(y_test, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_test, y_proba_rf))


              precision    recall  f1-score   support

           0       0.90      0.98      0.94       167
           1       0.79      0.45      0.58        33

    accuracy                           0.89       200
   macro avg       0.85      0.72      0.76       200
weighted avg       0.88      0.89      0.88       200

ROC AUC: 0.9138087461440755


In [25]:
from sklearn.ensemble import RandomForestClassifier


rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_model.fit(X_train_scaled, y_train)


threshold = 0.3
y_pred_rf_new = (y_proba_rf >= threshold).astype(int)

y_pred_rf = rf_model.predict(X_test_scaled)
y_proba_rf = rf_model.predict_proba(X_test_scaled)[:, 1]


print(classification_report(y_test, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_test, y_proba_rf))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94       167
           1       0.79      0.45      0.58        33

    accuracy                           0.89       200
   macro avg       0.85      0.72      0.76       200
weighted avg       0.88      0.89      0.88       200

ROC AUC: 0.9138087461440755
